In [1]:
#%%shell

#rm -rf ./torch-scae
#git clone https://github.com/bdsaglam/torch-scae
#cd ./torch-scae
#pip install -q .

In [2]:
%load_ext tensorboard

!mkdir /content/lightning_logs
%tensorboard --logdir '/content/lightning_logs'

The syntax of the command is incorrect.


Launching TensorBoard...

KeyboardInterrupt: 

In [1]:
import pathlib

import numpy as np
import torch
import torchvision
from torch_scae_experiments.boneage.train import train
from torch_scae_experiments.boneage.hparams import model_params
from pytorch_lightning import seed_everything

seed_everything(42)

42

In [2]:
from pprint import pprint

pprint(model_params)

{'image_shape': (3, 256, 256),
 'n_classes': 1,
 'n_obj_caps': 16,
 'n_part_caps': 30,
 'ocae_decoder_capsule_params': {'allow_deformations': True,
                                 'caps_dropout_rate': 0.0,
                                 'dim_caps': 32,
                                 'hidden_sizes': (128,),
                                 'learn_vote_scale': True,
                                 'noise_scale': 4.0,
                                 'noise_type': 'uniform',
                                 'similarity_transform': False},
 'ocae_encoder_set_transformer_params': {'dim_hidden': 16,
                                         'dim_out': 256,
                                         'layer_norm': True,
                                         'n_heads': 1,
                                         'n_layers': 3},
 'pcae_cnn_encoder_params': {'activate_final': True,
                             'kernel_sizes': [3, 3, 3, 3],
                             'out_channels': [128, 

In [3]:
# Customize model parameters

# model_params.update(dict(
#     n_part_caps=36,
#     n_obj_caps=10,
#     pcae_template_generator_params=dict(
#         template_size=(11, 11),
#         template_nonlin='sigmoid',
#         colorize_templates=False,
#     ),
#     scae_params=dict(
#         posterior_sparsity_loss_type='entropy',
#         posterior_within_example_sparsity_weight=1,
#         posterior_between_example_sparsity_weight=1,
#     )
# ))

In [4]:
training_params = dict(
    data_dir=str(pathlib.Path.home() / 'torch-scae/data/boneage'),
    gpus=1,
    batch_size=16,
    num_workers=2,
    max_epochs=100,
    learning_rate=3e-5,
    optimizer_type='RMSprop',
    use_lr_scheduler=True,
    lr_scheduler_decay_rate=0.997
)

In [5]:
torch.cuda.reset_max_memory_cached()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_accumulated_memory_stats()

train(model_params, **training_params)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]

    | Name                                              | Type                      | Params
--------------------------------------------------------------------------------------------
0   | scae                                              | SCAE                      | 2 M   
1   | scae.part_encoder                                 | CapsuleImageEncoder       | 984 K 
2   | scae.part_encoder.encoder                         | CNNEncoder                | 446 K 
3   | scae.part_encoder.encoder.network                 | Sequential                | 446 K 
4   | scae.part_encoder.encoder.network.0               | Conv2d                    | 3 K   
5   | scae.part_encoder.encoder.network.1               | ReLU                      | 0     
6   | scae.part_encoder.encoder.network.2               | Conv2d                    | 147 K 
7   | scae.part_encoder.encoder.network.3      

RuntimeError: CUDA out of memory. Tried to allocate 372.00 MiB (GPU 0; 4.00 GiB total capacity; 2.10 GiB already allocated; 238.35 MiB free; 2.70 GiB reserved in total by PyTorch)

In [ ]:
# resume training
training_params.update(
    max_epochs=200,
    resume_from_checkpoint= '/content/lightning_logs/version_0/checkpoints/epoch=99.ckpt')
)
train(model_params, **training_params)